In [ ]:
import os
import pandas as pd
import re
import math
import copy
import sys
# Add the root directory /workspaces/llm_etl to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))
# Now import your module
from spider2_utils import load_csv_database


-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("bank_sales_trading", rows_limit=-1)

interest_metrics = _database["interest_metrics"]
interest_map = _database["interest_map"]

### Question
Identify the top 10 and bottom 10 interest categories based on their highest composition values across all months. For each category, display the time(MM-YYYY), interest name, and the composition value

### User Intent 2: Filter out rows from `interest_metrics` where `month_year` is NULL.

In [ ]:
interest_metrics_filtered = interest_metrics[interest_metrics['month_year'].notnull()].copy()

### User Intent 3: Join `interest_metrics_filtered` with `interest_map` on `interest_id` = `id`.

In [ ]:
joined_df = interest_metrics_filtered.merge(interest_map, left_on='interest_id', right_on='id', how='inner')

### User Intent 4: Rank compositions within each `interest_name` group in descending order.

In [ ]:
joined_df['interest_rank'] = joined_df.groupby('interest_name')['composition'].rank(method='min', ascending=False)

### User Intent 5: Filter rows where `interest_rank` is 1.

In [ ]:
top_rank_df = joined_df[joined_df['interest_rank'] == 1].copy()

### User Intent 6: Select columns `month_year`, `interest_name`, `composition` for further ranking.

In [ ]:
top_rank_df = top_rank_df[['month_year', 'interest_name', 'composition']].copy()

### User Intent 7: Extract top 10 rows by `composition` in descending order.

In [ ]:
top_10 = top_rank_df.sort_values(by='composition', ascending=False).head(10)

### User Intent 8: Extract bottom 10 rows by `composition` in ascending order.

In [ ]:
bottom_10 = top_rank_df.sort_values(by='composition', ascending=True).head(10)

### User Intent 9: Combine top 10 and bottom 10 rows into one DataFrame.

In [ ]:
combined = pd.concat([top_10, bottom_10], axis=0)

### User Intent 10: Sort the final result by `composition` in descending order.

In [ ]:
final_result = combined.sort_values(by='composition', ascending=False).reset_index(drop=True)
final_result